In [1]:
# import libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras

2022-11-30 22:32:13.666383: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.11.0'

## Convert model from Keras to TF-Lite format

In [3]:
# load model
model = keras.models.load_model('dino_dragon_10_0.899.h5')

In [4]:
# convert

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('dino_dragon_tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/q6/4vk5yrts387cp_3qs430rhqc0000gn/T/tmpwn4a10jm/assets


INFO:tensorflow:Assets written to: /var/folders/q6/4vk5yrts387cp_3qs430rhqc0000gn/T/tmpwn4a10jm/assets
2022-11-30 22:32:50.912583: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-30 22:32:50.912833: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.


In [5]:
!ls -lh

total 262984
-rw-r--r--@ 1 pllee  staff    86M Nov 30 14:28 dino_dragon_10_0.899.h5
-rw-r--r--  1 pllee  staff    43M Nov 30 22:32 dino_dragon_tflite
-rw-r--r--  1 pllee  staff    14K Nov 30 22:30 hw9.ipynb
-rw-r--r--  1 pllee  staff   1.4K Nov 30 22:17 lambda_function.py


## Output index of the lite model

In [10]:
# Restart the kernel here to run tflite_runtime interpreter
import tflite_runtime.interpreter as tflite
import numpy as np

In [11]:
interpreter = tflite.Interpreter(model_path='dino_dragon_tflite')
interpreter.allocate_tensors()

interpreter.get_output_details()[0]['index']

13

## Prepare the image

In [12]:
!pip install pillow

In [13]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [14]:
# image url
url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'

In [15]:
# download image
img = download_image(url)

In [16]:
# prepare image
target_size = (150, 150)   # from homework 8
img = prepare_image(img, target_size)

/var/folders/q6/4vk5yrts387cp_3qs430rhqc0000gn/T/ipykernel_12259/2487457277.py:17: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.resize(target_size, Image.NEAREST)


In [17]:
# turn the image into numpy array and pre-process it
x = np.array(img, dtype='float32')
X = np.array([x])
X = X/255
X

array([[[[0.5529412 , 0.31764707, 0.1764706 ],
         [0.57254905, 0.34117648, 0.1764706 ],
         [0.5647059 , 0.3254902 , 0.14901961],
         ...,
         [0.07058824, 0.02745098, 0.01960784],
         [0.07058824, 0.03921569, 0.02745098],
         [0.10980392, 0.04705882, 0.04705882]],

        [[0.5137255 , 0.28235295, 0.14901961],
         [0.5529412 , 0.30980393, 0.15686275],
         [0.5803922 , 0.34901962, 0.18431373],
         ...,
         [0.05882353, 0.04313726, 0.03137255],
         [0.05490196, 0.02352941, 0.01176471],
         [0.09803922, 0.03529412, 0.03921569]],

        [[0.5058824 , 0.29411766, 0.15294118],
         [0.5686275 , 0.34117648, 0.19607843],
         [0.5568628 , 0.32156864, 0.1764706 ],
         ...,
         [0.07450981, 0.02745098, 0.02745098],
         [0.09019608, 0.03529412, 0.03137255],
         [0.07450981, 0.03529412, 0.03137255]],

        ...,

        [[0.29803923, 0.41960785, 0.36078432],
         [0.4       , 0.36078432, 0.3529412 ]

In [18]:
#  apply this model to this image. What's the output of the model?

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
input_index, output_index

(0, 13)

In [19]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
interpreter.get_tensor(output_index)

array([[0.8244865]], dtype=float32)

In [20]:
!docker images

REPOSITORY                             TAG           IMAGE ID       CREATED         SIZE
svizor42/zoomcamp-dino-dragon-lambda   v2            20ef58b21a05   12 days ago     639MB
hw5-cont                               latest        6aa48ab13cc5   7 weeks ago     587MB
<none>                                 <none>        7aa623322e98   7 weeks ago     587MB
svizor/zoomcamp-model                  3.9.12-slim   571a6fdc554b   8 weeks ago     125MB
sandbox_algod                          latest        9161df8c7986   5 months ago    1.32GB
sandbox_indexer                        latest        a11be98c98f8   6 months ago    801MB
postgres                               13-alpine     2557ebf9b7df   6 months ago    185MB
docker/getting-started                 latest        cb90f98fd791   7 months ago    28.8MB
volumes                                latest        ca3533566d5d   11 months ago   145MB
outyet                                 1.9.4-local   f25a7ea27eb1   11 months ago   11.7MB
leepat/b